# Webscrapping de l'indice VIX

## Obtenir la bonne page HTML avec Selenium

Avant de commencer le webscrapping, nous devons effectuer quelques modifications dans le code HMTML de base de la pgae web. Ainsi, pour obtenir le jeu de données entier, nous allons utiliser la librairie **selenium** qui nous permettra d'obtenir la page HTML avec toutes les données.

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC


In [34]:

driver = webdriver.Chrome(ChromeDriverManager().install())#,options=chrome_options)
driver.get('https://fr.investing.com/indices/cac-40-vix-historical-data')
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

#elements =driver.find_elements(By.CSS_SELECTOR,'css-1545rns-container')
dropdown =driver.find_element("id",'history-timeframe-selector')
dropdown.click() # on ouvre le dropdown

# on cherche les différentes options présentes dans le dropdown et on clique sur celle qu'on veut
driver.find_element("id","react-select-2-option-2").click() # 0 -> journalier, 1 -> Hebdomadaire, 2 -> pour Mensuel 


calendar=driver.find_element(By.CLASS_NAME,"historical-data_history-date-picker-wrapper__dDOuq")
calendar.click()

date_input = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div/div[2]/main/div/div[5]/div/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input')
date_input.clear()
date_input.send_keys("2021-12-11") #date à laquelle on souhaite commencer (marche pas mais permet de donner tout l'historique pour l'instant)


validate=driver.find_element(By.CLASS_NAME,"HistoryDatePicker_arrow-icon__NwxN4")
validate.click()

driver.find_element(By.CLASS_NAME,"inv-button signup_close__MFV87")
#WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME,"inv-button signup_close__MFV87"))).click() # on attend 10 secondes avant de break en l'abscence du bouton



webpage=driver.page_source # on stock le code html de la page où toutes le données sont chargées
#driver.quit()  



C:\Users\elelo\AppData\Local\Temp/ipykernel_352/2172984099.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())#,options=chrome_options)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".inv-button signup_close__MFV87"}
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x0058ACD3+2075859]
	Ordinal0 [0x0051EE61+1633889]
	Ordinal0 [0x0041B7BD+571325]
	Ordinal0 [0x0044AC2F+764975]
	Ordinal0 [0x0044AE1B+765467]
	Ordinal0 [0x0047D0F2+970994]
	Ordinal0 [0x00467364+881508]
	Ordinal0 [0x0047B56A+963946]
	Ordinal0 [0x00467136+880950]
	Ordinal0 [0x0043FEFD+720637]
	Ordinal0 [0x00440F3F+724799]
	GetHandleVerifier [0x0083EED2+2769538]
	GetHandleVerifier [0x00830D95+2711877]
	GetHandleVerifier [0x0061A03A+521194]
	GetHandleVerifier [0x00618DA0+516432]
	Ordinal0 [0x0052682C+1665068]
	Ordinal0 [0x0052B128+1683752]
	Ordinal0 [0x0052B215+1683989]
	Ordinal0 [0x00536484+1729668]
	BaseThreadInitThunk [0x7512FA29+25]
	RtlGetAppContainerNamedObjectPath [0x772A7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x772A7B8E+238]


## From HTML to Dataframe :

Maintenant que nous avons le code HTML de toutes les données, nous allons décortiquer la page grâce à la librairie BeautifulSoup.

In [4]:
import bs4
import pandas

In [5]:
# utiliser le package BeautifulSoup qui "comprend" les balises 

page = bs4.BeautifulSoup(webpage, "lxml")

print(page)

<html class="html" dir="ltr" lang="fr"><head><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-PGT3R3D&amp;l=dataLayerGTMParams"></script><script async="" gtm="GTM-PGT3R3D" src="https://www.googleoptimize.com/optimize.js?id=GTM-WW3TL3X&amp;l=dataLayerGTMParams" type="text/javascript"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-C4NDLGKVMK&amp;l=dataLayerGTMParams&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script>
                window.geo_country = 'FR';
                window.geo_state = 'IDF';
            </script>
<script charset="UTF-8" data-domain-script="265d5ac2-7d0c-4675-bf4d-421a61fb482e" src="https://cookie-cdn.cookiepro.com/scripttemplates/otSDKStub.js" type="text/javascript"></script>
<meta charset="utf-8"/><meta content="initial-scale=1.0, width

In [6]:
# on identifie le tableau des indices vix : c'est le premier qui a cette classe "datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE"
tableau_indice = page.find('table', {'class' : 'datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE'})

print(tableau_indice)

<table class="datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE" data-test="historical-data-table" style="--freeze-column-width:100px;"><thead class="datatable_head__ysELt"><tr class="datatable_row__qHMpQ"><th class="datatable_cell__0y0eu"><div class="datatable_cell__wrapper__ddKL4"><button class="inv-button datatable_sort__vA7p1" type="button"><span>Date</span><span class="flex flex-col"><svg aria-hidden="true" class="datatable_sort-icon__vK_nK datatable_sort-icon-up__w5HvI" fill="none" style="height: auto;" viewbox="0 0 24 24" width="1em"><path d="M1 6L12 17L23 6H1Z" fill="currentColor"></path></svg><svg aria-hidden="true" class="datatable_sort-icon__vK_nK" fill="none" style="height: auto;" viewbox="0 0 24 24" width="1em"><path d="M1 6L12 17L23 6H1Z" fill="currentColor"></path></svg></span></button></div></th><th class="datatable_cell__0y0eu datatable_cell--align-end__fwomz"><div class="datatable_cell__wra

In [24]:
table_body = tableau_indice.find('tbody')

# on recherche toutes les lignes du tableau avec la balise "tr"
rows = table_body.find_all('tr')

# on obtient une liste où chaque élément est une des lignes du tableau



[<tr class="datatable_row__qHMpQ" data-test="historical-data-table-row"><td class="datatable_cell__0y0eu font-bold"><time datetime="01/04/2022">01/04/2022</time></td><td class="datatable_cell__0y0eu datatable_cell--align-end__fwomz datatable_cell--down__d61fR" dir="ltr">18,96</td><td class="datatable_cell__0y0eu datatable_cell--align-end__fwomz">18,96</td><td class="datatable_cell__0y0eu datatable_cell--align-end__fwomz">18,96</td><td class="datatable_cell__0y0eu datatable_cell--align-end__fwomz">18,96</td><td class="datatable_cell__0y0eu datatable_cell--align-end__fwomz" data-test="volume-cell"></td><td class="datatable_cell__0y0eu datatable_cell--align-end__fwomz font-bold" dir="ltr">0.00%</td></tr>,
 <tr class="datatable_row__qHMpQ" data-test="historical-data-table-row"><td class="datatable_cell__0y0eu font-bold"><time datetime="01/02/2022">01/02/2022</time></td><td class="datatable_cell__0y0eu datatable_cell--align-end__fwomz datatable_cell--down__d61fR" dir="ltr">18,96</td><td cla

In [8]:
cols = rows[1].find_all('td')
print(cols[0])
print(cols[0].text.strip())

<td class="datatable_cell__0y0eu font-bold"><time datetime="01/02/2022">01/02/2022</time></td>
01/02/2022


In [9]:
for ele in cols : 
    print(ele.text.strip())

01/02/2022
18,96
18,96
18,96
18,96

0.00%


Voici le dictionnaire de tous les indices VIX :

In [10]:
dico_vix = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_vix[cols[0]] = cols[1:]
dico_vix

{'01/04/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/02/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/01/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/11/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/10/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/09/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/08/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/07/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/03/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/01/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '01/12/2020': ['18,96', '22,22', '75,26', '13,89', '', '-14.68%'],
 '01/11/2020': ['22,22', '30,06', '48,12', '14,73', '', '-31.48%'],
 '01/10/2020': ['32,43', '23,07', '95,07', '16,51', '', '+29.14%'],
 '01/09/2020': ['25,11', '23,56', '101,00', '19,11', '', '+14.64%'],
 

In [19]:
data_vix = pandas.DataFrame.from_dict(dico_vix,orient='index')
data_vix.head()

,0,1,2,3,4,5
01/04/2022,"18,96","18,96","18,96","18,96",,0.00%
01/02/2022,"18,96","18,96","18,96","18,96",,0.00%
01/01/2022,"18,96","18,96","18,96","18,96",,0.00%
01/12/2021,"18,96","18,96","18,96","18,96",,0.00%
01/11/2021,"18,96","18,96","18,96","18,96",,0.00%


In [28]:
table_head=page.find("thead",{'class': 'datatable_head__ysELt'})
col_name_head=[]
for x in table_head.find_all("tr"):
    
    for y in x.find_all('th'):
        col_name_head.append(y.text)
col_name_head

['Portefeuille']

In [20]:
data_vix.columns=col_name_head[1:]

ValueError: Length mismatch: Expected axis has 6 elements, new values have 0 elements

In [21]:
data_vix

,0,1,2,3,4,5
01/04/2022,"18,96","18,96","18,96","18,96",,0.00%
01/02/2022,"18,96","18,96","18,96","18,96",,0.00%
01/01/2022,"18,96","18,96","18,96","18,96",,0.00%
01/12/2021,"18,96","18,96","18,96","18,96",,0.00%
01/11/2021,"18,96","18,96","18,96","18,96",,0.00%
...,...,...,...,...,...,...
01/06/2000,"23,15","25,86","25,97","20,78",,-16.52%
01/05/2000,"27,73","25,47","32,05","25,27",,+5.48%
01/04/2000,"26,29","27,83","30,35","21,56",,-2.67%
01/03/2000,"27,01","27,81","28,17","22,38",,-5.59%


In [89]:
from selenium.webdriver.support.ui import Select
import time

# Les deux lignes suivantes pemettent d'empêcher l'apparition de pop up (cookies, agreement...) sur la page
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})

driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
driver.get('https://www.boursorama.com/bourse/indices/cours/historique/1cVIX')
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))).click()
#WebDriverWait(driver, 10).until(EC.visibility_of((By.ID, "historic_search[duration]")))

#select = Select(driver.find_element(By.ID,'historic_search[duration]'))
#select.select_by_visible_text("3 ans")


#element=WebDriverWait(driver,30).until(EC.presence_of_element_located((By.CSS_SELECTOR,"option[value='1']")))
#select=Select(driver.find_element(By.ID,'historic_search_period'))

#select.select_by_visible_text("Hebdo")

from selenium.common.exceptions import TimeoutException

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "option[value='7']"))
    )
    print("Option loaded")
except TimeoutException:
    print("Time exceeded!")

select=Select(driver.find_element(By.ID,'historic_search_period'))

select.select_by_visible_text("Hebdo")




C:\Users\elelo\AppData\Local\Temp/ipykernel_17784/3602946800.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)


Option loaded


ElementNotInteractableException: Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=107.0.5304.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x009FACD3+2075859]
	Ordinal0 [0x0098EE61+1633889]
	Ordinal0 [0x0088B7BD+571325]
	Ordinal0 [0x0088E374+582516]
	Ordinal0 [0x0088E225+582181]
	Ordinal0 [0x0088E4C0+582848]
	Ordinal0 [0x008BE7BE+780222]
	Ordinal0 [0x008B2166+729446]
	Ordinal0 [0x008D731C+881436]
	Ordinal0 [0x008B15BF+726463]
	Ordinal0 [0x008D7534+881972]
	Ordinal0 [0x008EB56A+963946]
	Ordinal0 [0x008D7136+880950]
	Ordinal0 [0x008AFEFD+720637]
	Ordinal0 [0x008B0F3F+724799]
	GetHandleVerifier [0x00CAEED2+2769538]
	GetHandleVerifier [0x00CA0D95+2711877]
	GetHandleVerifier [0x00A8A03A+521194]
	GetHandleVerifier [0x00A88DA0+516432]
	Ordinal0 [0x0099682C+1665068]
	Ordinal0 [0x0099B128+1683752]
	Ordinal0 [0x0099B215+1683989]
	Ordinal0 [0x009A6484+1729668]
	BaseThreadInitThunk [0x7512FA29+25]
	RtlGetAppContainerNamedObjectPath [0x7